# Import Necessary Libraries

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict

import warnings
warnings.filterwarnings('ignore')

# Load the Train Test Split

In [26]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12) 
Testing Samples : (83, 250, 12)
Training Labels : (332,) 
Testing Labels : (83,)


In [27]:
_ , num_timeframes, num_features = X_train.shape

# Reshape data for the model
-----------------
Samples are a 2D matrix which need to be reshaped to 1D to feed into the model

In [28]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 3000) 
Testing Samples : (83, 3000)
Training Labels : (332,) 
Testing Labels : (83,)


# Grid search over SVM_OVO model

In [29]:
parameters = {
    "kernel": ['rbf','poly'],
    "C": np.logspace(0, 4, 5), 
    "degree": [5,6,7]
}

grid = GridSearchCV(SVC(decision_function_shape='ovo', gamma='scale'), parameters, cv=5, verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] C=1.0, degree=5, kernel=rbf .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.4s
[CV] C=1.0, degree=5, kernel=rbf .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.4s
[CV] C=1.0, degree=5, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.4s
[CV] C=1.0, degree=5, kernel=poly ....................................
[CV] ..................... C=1.0, degree=5, kernel=poly, total=   0.4s
[CV] C=1.0, degree=5, kernel=poly ....................................
[CV] ..................... C=1.0, degree=5, kernel=poly, total=   0.4s
[CV] C=1.0, degree=5, kernel=poly ....................................
[CV] ..................... C=1.0, degree=5, kernel=poly, total=   0.4s
[CV] C=1.0, degree=6, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=6, kernel=rbf, total=   0.4s
[CV] C=1.0, degree=6, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=6, kernel=rbf, total=   0.4s
[CV] C=1.0, degree=6, kernel=rbf .....................................
[CV] .

[CV] .................. C=1000.0, degree=5, kernel=poly, total=   0.3s
[CV] C=1000.0, degree=6, kernel=rbf ..................................
[CV] ................... C=1000.0, degree=6, kernel=rbf, total=   0.3s
[CV] C=1000.0, degree=6, kernel=rbf ..................................
[CV] ................... C=1000.0, degree=6, kernel=rbf, total=   0.3s
[CV] C=1000.0, degree=6, kernel=rbf ..................................
[CV] ................... C=1000.0, degree=6, kernel=rbf, total=   0.3s
[CV] C=1000.0, degree=6, kernel=poly .................................
[CV] .................. C=1000.0, degree=6, kernel=poly, total=   0.4s
[CV] C=1000.0, degree=6, kernel=poly .................................
[CV] .................. C=1000.0, degree=6, kernel=poly, total=   0.4s
[CV] C=1000.0, degree=6, kernel=poly .................................
[CV] .................. C=1000.0, degree=6, kernel=poly, total=   0.4s
[CV] C=1000.0, degree=7, kernel=rbf ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:   30.6s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovo', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04]),
                         'degree': [5, 6, 7], 'kernel': ['rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [30]:
params = grid.best_params_
best_estimator = grid.best_estimator_
print(params)
print(best_estimator)

{'C': 100.0, 'degree': 5, 'kernel': 'rbf'}
SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=5, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [31]:
clf = best_estimator
clf.fit(X_train,Y_train)

SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=5, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [32]:
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.81      0.93      0.87        14
           1       0.92      0.86      0.89        14
           2       1.00      0.92      0.96        13
           3       0.94      0.94      0.94        16
           4       1.00      0.86      0.92        14
           5       0.86      1.00      0.92        12

    accuracy                           0.92        83
   macro avg       0.92      0.92      0.92        83
weighted avg       0.92      0.92      0.92        83

[[13  0  0  1  0  0]
 [ 2 12  0  0  0  0]
 [ 1  0 12  0  0  0]
 [ 0  1  0 15  0  0]
 [ 0  0  0  0 12  2]
 [ 0  0  0  0  0 12]]


In [33]:
pickle.dump(clf, open(os.path.join("..", "IPD", "SVM_model_half.pkl"), 'wb'))
print("Model Saved")

Model Saved


# Train on entire data

In [39]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))

X_train = np.concatenate((X_train,X_test), axis=0)
Y_train = np.concatenate((Y_train,Y_test), axis=0)

_ , num_timeframes, num_features = X_train.shape
X_train = X_train.reshape((-1,num_timeframes*num_features))
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(X_train)

from sklearn.utils import shuffle
X_train, X_sparse, Y_train = shuffle(X_train, X_sparse, Y_train)
X_train = X_train.reshape((-1,num_timeframes,num_features))

In [40]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (415, 3000) 
Testing Samples : (83, 250, 12)
Training Labels : (415,) 
Testing Labels : (83,)


In [41]:
parameters = {
    "kernel": ['rbf','poly'],
    "C": np.logspace(0, 4, 10), 
    "degree": [5,6,7]
}

grid = GridSearchCV(SVC(decision_function_shape='ovo', gamma='scale'), parameters, cv=10, verbose=2)
grid.fit(X_train, Y_train)

Fitting 10 folds for each of 60 candidates, totalling 600 fits
[CV] C=1.0, degree=5, kernel=rbf .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.8s
[CV] C=1.0, degree=5, kernel=rbf .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.7s
[CV] C=1.0, degree=5, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.8s
[CV] C=1.0, degree=5, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.8s
[CV] C=1.0, degree=5, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.8s
[CV] C=1.0, degree=5, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.8s
[CV] C=1.0, degree=5, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.7s
[CV] C=1.0, degree=5, kernel=rbf .....................................
[CV] ...................... C=1.0, degree=5, kernel=rbf, total=   0.7s
[CV] C=1.0, degree=5, kernel=rbf .....................................
[CV] .

...

[CV] ....... C=1291.5496650148827, degree=7, kernel=rbf, total=   0.5s
[CV] C=1291.5496650148827, degree=7, kernel=rbf ......................
[CV] ....... C=1291.5496650148827, degree=7, kernel=rbf, total=   0.5s
[CV] C=1291.5496650148827, degree=7, kernel=rbf ......................
[CV] ....... C=1291.5496650148827, degree=7, kernel=rbf, total=   0.6s
[CV] C=1291.5496650148827, degree=7, kernel=rbf ......................
[CV] ....... C=1291.5496650148827, degree=7, kernel=rbf, total=   0.5s
[CV] C=1291.5496650148827, degree=7, kernel=rbf ......................
[CV] ....... C=1291.5496650148827, degree=7, kernel=rbf, total=   0.5s
[CV] C=1291.5496650148827, degree=7, kernel=poly .....................
[CV] ...... C=1291.5496650148827, degree=7, kernel=poly, total=   0.6s
[CV] C=1291.5496650148827, degree=7, kernel=poly .....................
[CV] ...... C=1291.5496650148827, degree=7, kernel=poly, total=   0.6s
[CV] C=1291.5496650148827, degree=7, kernel=poly .....................
[CV] .

[CV] ........ C=3593.813663804626, degree=7, kernel=rbf, total=   0.5s
[CV] C=3593.813663804626, degree=7, kernel=rbf .......................
[CV] ........ C=3593.813663804626, degree=7, kernel=rbf, total=   0.5s
[CV] C=3593.813663804626, degree=7, kernel=rbf .......................
[CV] ........ C=3593.813663804626, degree=7, kernel=rbf, total=   0.5s
[CV] C=3593.813663804626, degree=7, kernel=rbf .......................
[CV] ........ C=3593.813663804626, degree=7, kernel=rbf, total=   0.5s
[CV] C=3593.813663804626, degree=7, kernel=rbf .......................
[CV] ........ C=3593.813663804626, degree=7, kernel=rbf, total=   0.6s
[CV] C=3593.813663804626, degree=7, kernel=rbf .......................
[CV] ........ C=3593.813663804626, degree=7, kernel=rbf, total=   0.5s
[CV] C=3593.813663804626, degree=7, kernel=rbf .......................
[CV] ........ C=3593.813663804626, degree=7, kernel=rbf, total=   0.5s
[CV] C=3593.813663804626, degree=7, kernel=poly ......................
[CV] .

[CV] .................. C=10000.0, degree=7, kernel=rbf, total=   0.5s
[CV] C=10000.0, degree=7, kernel=rbf .................................
[CV] .................. C=10000.0, degree=7, kernel=rbf, total=   0.5s
[CV] C=10000.0, degree=7, kernel=rbf .................................
[CV] .................. C=10000.0, degree=7, kernel=rbf, total=   0.5s
[CV] C=10000.0, degree=7, kernel=rbf .................................
[CV] .................. C=10000.0, degree=7, kernel=rbf, total=   0.5s
[CV] C=10000.0, degree=7, kernel=rbf .................................
[CV] .................. C=10000.0, degree=7, kernel=rbf, total=   0.5s
[CV] C=10000.0, degree=7, kernel=rbf .................................
[CV] .................. C=10000.0, degree=7, kernel=rbf, total=   0.5s
[CV] C=10000.0, degree=7, kernel=rbf .................................
[CV] .................. C=10000.0, degree=7, kernel=rbf, total=   0.5s
[CV] C=10000.0, degree=7, kernel=rbf .................................
[CV] .

[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  6.4min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovo', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                         'degree': [5, 6, 7], 'kernel': ['rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [42]:
clf = grid.best_estimator_
clf.fit(X_train,Y_train)

SVC(C=464.15888336127773, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovo', degree=5, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

# Saving model

In [43]:
pickle.dump(clf, open(os.path.join("..", "models", "SVM_model.pkl"), 'wb'))
print("Model Saved")

Model Saved
